In [78]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import re
import nltk

In [79]:
def read_text (file):
    with open(file,'r') as f:
        text = f.read()
    f.close()
    return text

In [80]:
bb = read_text('data/breitbart_pol_feb_20_26.txt')
nyt = read_text('data/nyt_pol_feb_20_26.txt')
hp = read_text('data/huffpost_pol_feb_20_26.txt')
fox = read_text('data/fox_pol_feb_20_26.txt')


In [81]:
d = [bb,nyt,fox,hp]
names = ["breitbart","nyt","fox","hp"]

In [82]:
stopWords = stopwords.words('english')
tokenize = lambda doc: list(filter(None,re.split(r"\W|\d",doc.lower())))


In [98]:
tfidf_vec = TfidfVectorizer(stop_words = stopWords,tokenizer=tokenize)
tfidf = tfidf_vec.fit_transform(d)
#print(td_mat.shape)
#count_vect.get_feature_names()

In [99]:
#tf and tf-idf
# tf_transformer = TfidfTransformer(use_idf=False)
# tf = tf_transformer.fit_transform(td_mat)
# tfidf_transformer = TfidfTransformer()
# tfidf = tfidf_transformer.fit_transform(td_mat)

In [100]:
from sklearn.metrics.pairwise import linear_kernel

In [101]:
def cosine_similarities_mat(m):
    n = m.shape[0]
    cs_array = np.zeros((n,n))
    for i in range(n):
        cs_array[i]=(linear_kernel(m[i:i+1], m).flatten())
    return cs_array

In [102]:
cosine_similarities_mat(tfidf)

array([[ 1.        ,  0.59833811,  0.80777284,  0.81084176],
       [ 0.59833811,  1.        ,  0.63252188,  0.63501316],
       [ 0.80777284,  0.63252188,  1.        ,  0.84319142],
       [ 0.81084176,  0.63501316,  0.84319142,  1.        ]])

In [104]:
def pearson_corr_mat(m):
    n = m.shape[0]
    pc_array_all = np.corrcoef(m.toarray(), m.toarray())
    pc_array = np.zeros((n,n))
    for i in range(n):
        pc_array[i]=pc_array_all[i][0:4]
    return pc_array

In [105]:
pearson_corr_mat(tfidf)

array([[ 1.        ,  0.57311568,  0.79083545,  0.78999278],
       [ 0.57311568,  1.        ,  0.61108668,  0.61226321],
       [ 0.79083545,  0.61108668,  1.        ,  0.82938021],
       [ 0.78999278,  0.61226321,  0.82938021,  1.        ]])

In [12]:
count_vect = CountVectorizer(max_features = 50,stop_words = stopWords,tokenizer=tokenize)
td_mat = count_vect.fit_transform(d)
#count_vect.get_feature_names()
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(td_mat)

In [106]:
def jaccard_similarity(d1, d2):
    intersection = set(d1).intersection(set(d2))
    union = set(d1).union(set(d2))
    return len(intersection)/len(union)

In [107]:
from nltk.probability import FreqDist

In [108]:
d_tokenized =[tokenize(doc) for doc in d]
d_no_stop =[[w for w in d if w not in stopWords] for d in d_tokenized]
d_Freq =[FreqDist(doc) for doc in d_no_stop]
d_common_50 =[f.most_common(50) for f in d_Freq]

In [109]:
d_most_common_words = [list(zip(*d))[0] for d in d_common_50]

In [111]:
def jaccard_similarity_mat(m):
    n = len(m)
    array = np.zeros((n,n))
    for i in range(n):
        for j in range(i,n):
            sim = jaccard_similarity(m[i],m[j])
            array[i][j] = sim
            array[j][i] = sim
    return array

In [112]:
jaccard_similarity_mat(d_most_common_words)

array([[ 1.        ,  0.2987013 ,  0.36986301,  0.33333333],
       [ 0.2987013 ,  1.        ,  0.36986301,  0.35135135],
       [ 0.36986301,  0.36986301,  1.        ,  0.42857143],
       [ 0.33333333,  0.35135135,  0.42857143,  1.        ]])